# Materials design by *ab initio* methods (MDAM)

Prof. Blazej Grabowski, Dr. Yuji Ikeda

Department of Materials Design, University of Stuttgart

# Class work 02: Schrödinger equation in 1D real space

In [ ]:
import numpy as np

## Eigenproblem with NumPy

In quantum-mechanics-based simulations, including *ab initio* calculations, eigenvalue problems are the most important to understand the physics.
An eigenvalue $\lambda$ and the corresponding eigenvector $\mathbf{x}$ of a matrix $A$ satisfy

\begin{align}
A \mathbf{x} = \lambda \mathbf{x}.
\end{align}

Since in most *ab initio* calculations $A$ is Hermitian, i.e., $A^\dagger = A$, we focus on Hermitian matrices.
If all the elements are real, the Hermitian matrix is actually symmetric, i.e., $A^\mathrm{T} = A$.

In NumPy, we can easily solve the eigenproblem of an Hermitian matrix using [`np.linalg.eigh`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html). Let's do this for the matrix:

$$
A = 
\begin{pmatrix}
 2 &  2 & -1 \\
 2 & -1 &  2 \\
-1 &  2 &  2 \\
\end{pmatrix}.
$$

In [ ]:
A = [
    [2, 2, -1],
    [2, -1, 2],
    [-1, 2, 2],
]
w, v = np.linalg.eigh(A)   # w are the eigenvalues, v are the eigenvectors
w, v

**All the eigenvalues of a Hermitian matrix are real.**

In the `np.linalg.eigh` method, the computed eigenvalues are already sorted in ascending order.

Let's check whether the obtained eigenvalues and eigenvectors satisfy the equation above.

In [ ]:
i = 2  # Check the first eigenstate
print(A @ v[:, i])  # LHS
print(w[i] * v[:, i])  # RHS

The operator `@` is for the matrix product (`A @ B`). You can also use `np.dot(A, B)`.

**Eigenvectors of a Hermitian matrix are orthonormalized**, i.e.,

In [ ]:
print(v[:, 0] @ v[:, 0])  # substantially 1
print(v[:, 0] @ v[:, 1])  # substantially 0

We also notice that, among the three eigenvalues, two are the same.
We refer to such a case as "**two eigenstates are degenerate**".
Then, any arbitrary linear combination of the degenerate eigenvectors is also an eigenvector of the corresponding eigenvalue.
Let's confirm this.

In [ ]:
# Take a certain linear combination of the two eigenvectors.
w_new = w[1]
v_new = 2.0 * v[:, 1] + v[:, 2]
print(A @ v_new)
print(w_new * v_new)

## ***Problem:*** Pauli matrices

Compute the eigenvalues and eigenvectors of the following [Pauli matrices](https://en.wikipedia.org/wiki/Pauli_matrices) and do a similar check as above:

\begin{align}
\sigma_1 =
\begin{pmatrix}
0 & 1 \\
1 & 0 \\
\end{pmatrix},\quad
\sigma_2 =
\begin{pmatrix}
0 & -i \\
i &  0 \\
\end{pmatrix},\quad
\sigma_3 =
\begin{pmatrix}
1 &  0 \\
0 & -1 \\
\end{pmatrix}.
\end{align}

Hint: In Python, a complex number, e.g., 2 + 1*i* is expressed as `2 + 1j`, where `j` indicates the imaginary unit.

In [ ]:
# to_be_filled
sigma1 = [[0, 1], [1, 0]]
sigma2 = [[0, -1j], [1j, 0]]
sigma3 = [[1, 0], [0, -1]]

In [ ]:
# to_be_filled
values, vectors = np.linalg.eigh(sigma1)

In [ ]:
# to_be_filled
i = 1
print(sigma1 @ vectors[:, i])
print(values[i] * vectors[:, i])

In [ ]:
# to_be_filled
np.linalg.eigh(sigma2)

In [ ]:
# to_be_filled
np.linalg.eigh(sigma3)

## Managing data in `dataclass`

Before we dive into solving our first Schrödinger equations, we introduce a programming technique to limit the [scope](https://en.wikipedia.org/wiki/Name_collision) of variable names and to thus avoid [name collisions](https://en.wikipedia.org/wiki/Name_collision).

In Jupyter Notebook, the same variables can be reused in many places. This however causes problems when we want to rerun some parts of the Notebook; redefined variables might be used in unexpected places, which breaks the reproducibility of the results.

One smart and common way to avoid this issue is to define variables in a limited name "scope". Even if we use the same variable names, they do not conflict if they belong to different "scopes". This allows us to nicely handle variables that reappear in many places but specific to a certain example (or problem or task in your homework).
Utilizing such problem specific variables makes the output of the cells less sensitive to the order of execution, in particular when we come to interactive output controlled by sliders (ipywidgets).  

For this purpose, we use Python [`dataclass`](https://docs.python.org/3/library/dataclasses.htmls) in our exercises.

In [ ]:
from dataclasses import dataclass

Using `dataclass`, we define a class, which we call `SystemXXX`.
In there we will collect all relevant quantities that define the system we would like to solve.

In [ ]:
@dataclass
class System1D:
    """
    Dataset to solve the Schrödinger equation.

    Notes
    -----
    All the quantities are in Hartree atomic units (hbar = m_e = e = 1.0).
    """
    xmin: float  # first mesh point
    xmax: float  # last mesh point
    nx: int  # number of mesh points
    mass: float = 1.0  # mass (default: 1.0 for an electron)
    charge: float = -1.0  # charge (default: -1.0 for an electron)
    periodic: bool = False  # whether the system is periodic

    def __post_init__(self):
        # If periodic, `xmax` is excluded from the mesh points.
        endpoint = not self.periodic
        self.xs = np.linspace(self.xmin, self.xmax, self.nx, endpoint=endpoint)
        self.ax = self.xmax - self.xmin
        self.deltax = self.xs[1] - self.xs[0]

`@dataclass` makes a special class that smartly initialize the attributes.
(See [here](https://docs.python.org/3/library/dataclasses.html) for details.)

To make use of this class we assign it to a symbol, the name of which is example specific. Here, we just call it `s`; below we will add labels to get problem-specific symbols.

To initialize `s`, we have to provide three variables, `xmin`, `xmax`, and `nx`, that characterize our system.

In [ ]:
s = System1D(xmin=0.0, xmax=1.0, nx=16)

We can access the attribute of this object with a following dot, e.g.,

In [ ]:
s.xmin

Likewise, we can access other attributes initialized in the special method `__post_init__` (see details [here](https://docs.python.org/3/library/dataclasses.html#post-init-processing)).

In [ ]:
s.xs

We can add more variables to our class that are not in the original definition of the class.

Let's add, e.g., a quadratic potential `vs` into our variable collection:

In [ ]:
s.vs = s.xs**2

`s` contains now an additional variable, an array called `vs`. We can access it as before for the other variables:

In [ ]:
s.vs

Let's plot it with matplotlib.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.plot(s.xs, s.vs)
plt.show()

We can have a look at all variables currently in our class by:

In [ ]:
vars(s)

## ***Problem:*** Create classes

### Instance without endpoint

Make a new instance `s_periodic` of the `System1D` class, where we **exclude** `xmax` from the mesh points.

- You need to override the default value for the variable `periodic`.

Inspect the resulting mesh and delta and explain what has changed with respect to before.

In [ ]:
# to_be_filled
s_periodic = System1D(xmin=-1.0, xmax=+1.0, nx=16, periodic=True)
s_periodic.xs

In [ ]:
# to_be_filled
s_periodic.deltax

### Symmetric mesh around 0

Make a new instance `s_symmetric` of the `System1D` class that has a **symmetric** mesh around 0 from -2 bohr to +2 bohr.

Then add an attribute for a quadratic potential `s_symmetric.vs` to your class and plot it together with the potential we have seen above for `s`.

In [ ]:
# to_be_filled
s_symmetric = System1D(xmin=-2.0, xmax=2.0, nx=16)
s_symmetric.vs = s_symmetric.xs**2

In [ ]:
# to_be_filled
fig, ax = plt.subplots()
ax.plot(s.xs, s.vs, '.')
plt.plot(s_symmetric.xs, s_symmetric.vs)
plt.show()

## Schrödinger equation in the real-space-grid method
    
Now let's get to the real stuff. The Schrödinger equation is the eigenvalue equation of the Hamilton operator,

$$
\hat{H}\left|\psi\right\rangle =E\left|\psi\right\rangle.
$$

More concretely, this is the time-independent Schrödinger eqn. with $\hat{H}$ the Hamilton operator, $E$ the eigenvalue (energy), and $\psi$ the wave vector (ket vector in Hilbert space).

For a single-particle in 1D (no spin) 
$$
\hat{H} = \frac{\hat{p}{}^{2}}{2m}+\hat{V}(x)
$$
corresponding to the kinetic and potential energies. Thus we get

$$
\left(\frac{\hat{p}{}^{2}}{2m}+\hat{V}(x)\right)\left|\psi\right\rangle = E \left|\psi\right\rangle.
$$

Choosing the position space as the representation to solve the above numerically, we pre-multiply by the bra-vector $\left\langle x \right|$:

$$
\langle x | \left(\frac{\hat{p}{}^{2}}{2m}+\hat{V}(x)\right) | \psi \rangle
= \langle x | E | \psi \rangle
$$

(Check the lecture notes for more details on representations and how to go from the abstract Hilbert space to the real space.)

This leads to

$$
  \left(-\frac{\hbar^{2}}{2m}\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}}
+ V(x)\right)\psi(x)
= E\psi(x).
$$

To solve the above equation numerically, we need to discretize the real space into mesh points $x_{i}$ ($i = 0, \dots, n-1$).

The wave function and potential also become discrete vectors and multiplying $V(x_i)$ and $\psi(x_i)$ is also a discrete vector. 

To make sure it works, we represent $V$ as a $n \times n$ diagonal vector and $\psi$ as a single-column vector:

$$
V(x) \psi(x)
\quad\Rightarrow\quad
\begin{pmatrix}
V_{0}\\
 & V_{1}\\
 &  & V_{2}\\
 &  &  & \ddots\\
 &  &  &  &  V_{n-3}\\
 &  &  &  &  & V_{n-2}\\
 &  &  &  &  &  & V_{n-1}
\end{pmatrix}
\begin{pmatrix}
\psi_{0}\\
\psi_{1}\\
\psi_{2}\\
\vdots\\
\psi_{n-3}\\
\psi_{n-2}\\
\psi_{n-1}
\end{pmatrix}
=
\begin{pmatrix}
V_{1} \psi_{0}\\
V_{2} \psi_{1}\\
V_{3} \psi_{2}\\
\vdots\\
V_{n-2} \psi_{n-3}\\
V_{n-1} \psi_{n-2}\\
V_{n} \psi_{n-1}
\end{pmatrix}.
$$

The other term, the second derivative of the wave function, can also be *numerically approximated* using the finite-different method.

If the mesh points are equidistant,

$$
\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}}\psi(x)\bigg|_{x=x_i} = \frac{\psi_{i+1}-2\psi_{i}+\psi_{i-1}}{\Delta^{2}},
$$

where $\Delta$, the distance between two mesh points.

We also need to consider how to treat the endpoints. Here we consider the following **Dirichlet boundary conditions** (DBCs)

$$\psi_{-1} = \psi_{n} = 0.$$

In matrix notation, the kinetic energy term becomes:

$$
\hat{T} \psi(x)
=
-\frac{\hat{p}^2}{2m} \psi(x)
=
-\frac{\hbar^{2}}{2m}\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}} \psi(x)
\quad\Rightarrow\quad
-\frac{\hbar^2}{2 m \Delta^{2}}
\begin{pmatrix}
-2 &  1\\
 1 & -2 &  1\\
   &  1 & -2 & 1\\
   &    &  1 & \ddots & 1 \\
   &    &    &  1 & -2 &  1\\
   &    &    &    &  1 & -2 &  1\\
   &    &    &    &    &  1 & -2
\end{pmatrix}
\begin{pmatrix}
\psi_{0}\\
\psi_{1}\\
\psi_{2}\\
\vdots\\
\psi_{n-3}\\
\psi_{n-2}\\
\psi_{n-1}
\end{pmatrix}.
$$

Substituting both of the matrix notations into the real-space Schrödinger equation for a 1D particle, we get the final eigenvalue equation:

$$
\left[
-\frac{\hbar^{2}}{2m\Delta^{2}}
\begin{pmatrix}
-2 & 1\\
1 & -2 & 1\\
 & 1 & -2 & 1\\
 &  & 1 & \ddots & 1 \\
 &  &  &  1 & -2 &  1\\
 &  &  &    &  1 & -2 &  1\\
 &  &  &    &    &  1 & -2
\end{pmatrix}
+
\begin{pmatrix}
V_{0}\\
 & V_{1}\\
 &  & V_{2}\\
 &  &  & \ddots\\
 &  &  &  &  V_{n-3}\\
 &  &  &  &  & V_{n-2}\\
 &  &  &  &  &  & V_{n-1}
\end{pmatrix}
\right]
\begin{pmatrix}
\psi_{0}\\
\psi_{1}\\
\psi_{2}\\
\vdots\\
\psi_{n-3}\\
\psi_{n-2}\\
\psi_{n-1}
\end{pmatrix}
=
E
\begin{pmatrix}
\psi_{0}\\
\psi_{1}\\
\psi_{2}\\
\vdots\\
\psi_{n-3}\\
\psi_{n-2}\\
\psi_{n-1}
\end{pmatrix}.
$$

### Define the system

Let's solve the 1D Schrödinger equation numerically using the method mentioned above.

We first consider:

- an electron: *m* = *m*<sub>e</sub> and *q* = −*e*
- *x* ∈ [0, 1] (bohr)
- Dirichlet boundary conditions (DBCs)
- no potential: *V*(*x*) = 0

To handle the relevant system parameters, we make `s_DBC` using `System1D` defined above.

In [ ]:
s_DBC = System1D(xmin=0.0, xmax=1.0, nx=16, mass=1.0, charge=-1.0)

Note that `mass` and `charge` are optional; they are by default for an electron.

Keep also in mind that, throughout our own implementations, we use [Hartree atomic units](https://en.wikipedia.org/wiki/Hartree_atomic_units) $(m_\mathrm{e} = e = \hbar = 1)$.

### Calculate the kinetic-energy matrix

Next, we define a function `calc_T_1D(s)` to compute the kinetic-energy matrix.

This function takes the whole system class as its argument.
Specifically, we need:
- number of mesh points: `nx`
- distance between two mesh points: `deltax`
- mass of the particle: `mass`

In [ ]:
def calc_T_1D(s: System1D):
    """Calculate kinetic-energy matrix in Hartree atomic units."""
    # 2nd derivative
    T = np.eye(s.nx, k=+1) - 2.0 * np.eye(s.nx) + np.eye(s.nx, k=-1)
    T /= s.deltax**2

    # kinetic energy in Hartree atomic units (hbar = 1.0)
    T *= -1.0 / (2.0 * s.mass)

    return T

Store the kinetic energy matrix in a variable `s_DBC.T`:

In [ ]:
# kinetic energy
s_DBC.T = calc_T_1D(s_DBC)

### Calculate the potential-energy matrix

Similarly, store the potential-energy matrix in a variable `s_DBC.V`.

Since now *V*(*x*) = 0, the potential-energy matrix is just a zero matrix.

In [ ]:
# potential energy (zero)
s_DBC.V = np.zeros((s_DBC.nx, s_DBC.nx))

The Hamiltonian of the system (`s_DBC.H`) is the summation of `s_DBC.T` and `s_DBC.V`.

In [ ]:
s_DBC.H = s_DBC.T + s_DBC.V

### Plot matrices

To visualize the matrix, Matplotlib's [`ax.matshow`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.matshow.html) is convenient.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_T(s: System1D):
    vmax = np.abs(s.T).max()
    vmin = -vmax
    fig, ax = plt.subplots()
    pc = ax.matshow(s.T, cmap='bwr', vmin=vmin, vmax=vmax)
    fig.colorbar(pc)
    plt.show()

In [ ]:
plot_T(s_DBC)

### Solve eigenproblem

In [ ]:
s_DBC.eigvals, s_DBC.eigvecs = np.linalg.eigh(s_DBC.H)

### Inspecting eigenvalues

In [ ]:
def plot_eigvals(s: System1D):
    fig, ax = plt.subplots()
    ax.plot(s.eigvals, marker='o')
    ax.set_xlabel(r'Index $\nu$')
    ax.set_ylabel('Energy (hartree)')
    ax.set_title('Eigenvalues')
    plt.show()

In [ ]:
plot_eigvals(s_DBC)

Remember that the analytical eigenvalues of the Schrödinger equation are

$$
\epsilon_\nu = \frac{\nu^2 \pi^2 \hbar^2}{2 m a^2} \quad (\nu = 1, 2, \dots).
$$

Ref. https://en.wikipedia.org/wiki/Particle_in_a_box

Let's compare our numerical results with this analytical formula.

In [ ]:
nus = np.arange(1, s_DBC.nx + 1)  # [1, ..., s_DBC.nx]
coef = np.pi**2 / (2.0 * s_DBC.ax**2)
s_DBC.eigvals_analytical = coef * nus**2

In [ ]:
def plot_eigvals_with_analytical(s: System1D):
    fig, ax = plt.subplots()
    ax.plot(s.eigvals_analytical, 'k.', label='analytical')
    ax.plot(s.eigvals, '.', label='numerical')
    ax.set_xlabel(r'Index $\nu$')
    ax.set_ylabel('Energy (hartree)')
    ax.set_title('Eigenvalues')
    ax.legend()
    plt.show()

In [ ]:
plot_eigvals_with_analytical(s_DBC)

As can be seen, for small $\nu$, the numerical results nicely agree with the analytical values, but for large $k$, they deviate significantly. This is because (1) we have a discrete mesh and because (2) our kinetic energy is not exact but approximated by a second-order finite-difference. These issues can be solved, e.g., by using the plane-wave basis set. (We will do this in a later exercise.)

As seen, the eigenvalues lie on a sinusoidal curve (we will see the reason below).

### Normalize eigenfunctions

Let's plot the one-electron wavefunctions. Note that the wavefunction should be normalized to satisfy

$$
\int_0^a |\psi(x)|^2 \mathrm{d}x = \Delta \sum_{i=1}^{n} |\psi_i|^2 = 1,
$$

while the NumPy-computed eigenvectors $\mathbf{e}$ are normalized as

$$
\sum_{i=1}^{n} |e_i|^2 = 1.
$$

We therefore need to divide the computed eigenvectors by $\sqrt{\Delta}$ to satisfy the normalization condition for the wavefunction.
$$
\psi = \frac{e}{\sqrt{\Delta}}.
$$

In [ ]:
# normalization
s_DBC.psis = s_DBC.eigvecs / np.sqrt(s_DBC.deltax)

### Plot eigenfunctions

In [ ]:
def plot_psi(s: System1D, nu: int):
    psis = s.psis

    ymin = 1.1 * psis.min()
    ymax = 1.1 * psis.max()

    fig, ax = plt.subplots()

    ax.set_xlabel('$x$ (bohr)')
    ax.set_ylabel('$\psi(x)$')
    ax.plot(s.xs, psis[:, nu], '-o')
    ax.set_title(r'$\epsilon_\nu = $'+f'{s.eigvals[nu]} (hartree)')
    ax.set_ylim(ymin, ymax)

    plt.show()

In [ ]:
plot_psi(s_DBC, 0)

To plot the results interactively, the [`interactive`](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#interactive) method in `ipywidgets` is useful.

In [ ]:
from ipywidgets import interactive, fixed, IntSlider

The first argument for `interative` is the function that we would like to make interactive (`plot_psi`).

Be careful that what we need is the function itself rather than the returns of the function. Thus, the parentheses must not be added to `plot_psi` (No `plot_psi()`).

The following arguments are those to be given to `plot_psi`.

In the present case, `s` is not modified in the interactive plot; for such a variable, we should pass it via `fixed`.

The index of the eigenstates, `nu`, is what we would like to change. Since this is an integer value, we can make it interactive via `IntSlider` (possibly with some options like `max`).

In [ ]:
interactive(plot_psi, s=fixed(s_DBC), nu=IntSlider(max=s_DBC.nx-1))

## ***Problem:*** Interactive density plot

1. Calculate the probability densities $|\psi(x)|^2$ from `s_DBC.psis` and store it in `s_DBC.densities`.

2. Make an interactive plot of the probability density similar to the above.

3. Confirm that the probability density satisfies the normalization condition.

In [ ]:
# to_be_filled
s_DBC.densities = np.abs(s_DBC.psis)**2

In [ ]:
# to_be_filled
def plot_density(s: System1D, nu: int):
    densities = s.densities

    plt.xlabel('$x$ (bohr)')
    plt.ylabel('$|\psi(x)|^2$')
    plt.plot(s.xs, densities[:, nu], '-o')
    plt.title(r'$\epsilon_\nu = $'+f'{s.eigvals[nu]} (hartree)')
    plt.ylim(0, densities.max()+0.2)

    plt.show()

    print(np.sum(densities[:, nu]) * s.deltax)

In [ ]:
# to_be_filled
interactive(plot_density, s=fixed(s_DBC), nu=IntSlider(max=s_DBC.nx-1))

**Warning**: You will observe that the probability density becomes modestly smooth for the highest energy eigenstates.

However, this does **NOT** reflect the true physics **BUT** just a numerical issue.
As found from $\psi(x)$, we actually have severe fluctuations on the wavefunction, and the true density corresponding to the wavefunction should also show the same fluctuations.

The reason that the computed density appears to be smooth is simply because we do not have a mesh grid that is fine enough.
Actually, from the theory of discrete Fourier transforms, it is known that $|\psi(x)|^2$ requires a twice denser mesh than $\psi(x)$ to be correctly represented.

## ***Problem:*** 1D Coulomb potential

We next consider the system under the following 1D Coulomb potential.

$$
V(x) = -\frac{256}{|x|}
$$

(256 is just an arbitrary number.)

1. Define a system with $x$ in the range of [-4, +4] and with 64 mesh points.
2. Make a function `calc_vs_1D_Coulomb(s: System1D, X, Q, f)` to compute the following 1D Coulomb potential from a point charge $Q$ at the position $X$:

    $$
    V(x, q) \approx \frac{qQ}{|x - X| + f},
    $$

   where $x$ and $q$ are the position and the charge of the quantum particle to be solved, respectively.

   $f$ is a certain small number to avoid the singularity e.g. `1e-3`. (See also what happens if `f = 0.0`.)
   
3. Solve the eigenproblem and write functions to plot wavefunctions (`plot_psi_with_vs`) and probability densities (`plot_density_with_vs`), possibly together with the potential.

In [ ]:
# to_be_filled
s_Coulomb = System1D(xmin=-4.0, xmax=4.0, nx=64)

In [ ]:
# to_be_filled
def calc_vs_1D_Coulomb(s: System1D, X: float, Q: float, f: float = 1e-3):
    """Calculate potential in Hartree atomic units."""
    return s.charge * Q / (np.abs(s.xs - X) + f)

In [ ]:
# to_be_filled
s_Coulomb.vs = calc_vs_1D_Coulomb(s_Coulomb, X=0.0, Q=256.0)

In [ ]:
# to_be_filled
def plot_vs(s: System1D):
    plt.figure()
    plt.plot(s.xs, s.vs)
    plt.xlabel('$x$ (bohr)')
    plt.ylabel('$V(x)$ (hartree)')
    plt.show()

In [ ]:
# to_be_filled
plot_vs(s_Coulomb)

In [ ]:
# to_be_filled

# kinetic energy
s_Coulomb.T = calc_T_1D(s_Coulomb)

# potential energy
s_Coulomb.V = np.diag(s_Coulomb.vs)

# hamiltonian
s_Coulomb.H = s_Coulomb.T + s_Coulomb.V

# diagonalization
s_Coulomb.eigvals, s_Coulomb.eigvecs = np.linalg.eigh(s_Coulomb.H)

# normalization
s_Coulomb.psis = s_Coulomb.eigvecs / np.sqrt(s_Coulomb.deltax)

# densities
s_Coulomb.densities = np.abs(s_Coulomb.psis)**2

In [ ]:
# to_be_filled
def plot_psi_with_vs(nu: int, s: System1D):
    psis = s.psis

    ymin = 1.05 * psis.min()
    ymax = 1.05 * psis.max()

    fig, ax = plt.subplots()
    ax.plot(s.xs, psis[:, nu], 'o-')
    ax.set_xlabel('$x$ (bohr)')
    ax.set_ylabel('$\Psi(x)$')
    ax.set_ylim(ymin, ymax)
    ax.set_title(r'$\epsilon_\nu = $'+f'{s.eigvals[nu]} (hartree)')

    tax = plt.twinx()
    tax.plot(s.xs, s.vs, 'ko-')
    tax.set_ylabel('$V(x)$ (hartree)')

    plt.show()

In [ ]:
# to_be_filled
interactive(
    plot_psi_with_vs,
    s=fixed(s_Coulomb),
    nu=IntSlider(max=s_Coulomb.nx-1),
)

In [ ]:
# to_be_filled
def plot_density_with_vs(s: System1D, nu: int):
    densities = s.densities
    ymax = 1.05 * densities.max()

    fig, ax = plt.subplots()
    ax.plot(s.xs, densities[:, nu], 'o-')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$|\Psi(x)|^2$')
    ax.set_ylim(0.0, ymax)
    ax.set_title(r'$\epsilon_\nu = $'+f'{s.eigvals[nu]} (hartree)')

    tax = plt.twinx()
    tax.plot(s.xs, s.vs, 'ko-')
    tax.set_ylabel('$V(x)$ (hartree)')

    plt.show()

    print(sum(densities[:, nu]) * s.deltax)

In [ ]:
# to_be_filled
interactive(
    plot_density_with_vs,
    s=fixed(s_Coulomb),
    nu=IntSlider(max=s_Coulomb.nx-1),
)

## ***Problem:*** Periodic boundary conditions (PBCs)

Up to now we have considered electrons constrained in an isolated box (Dirichlet boundary condition (DBC)).
A crystal in the real world, however, has a tremendous number of electrons, in the order of 10<sup>23</sup> or higher.
Such a system cannot of course be simulated in the manner as we have done up to now.

To address this issue, we consider an infinitely large crystal, which is constructed by simply repeating a small *unit cell*.

Similarly to the above implementation with DBC, we discretize the unit cell into mesh points $x_{i}$ with $i$ from 0 to $n-1$.
The wave function and the potential are also discretized as $V_i = V(x_i)$ and $\psi_i = \psi(x_i)$.

Let's consider the Hamiltonian:
$$
\left(-\frac{\hbar^{2}}{2m}\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}}+V(x)\right)\psi(x) = E\psi(x).
$$

As we learned already, the second derivative of the wave function can be numerically approximated by the finite-difference method:

$$
\left.\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}}\psi(x)\right|_{x = x_i} =
\frac{\psi_{i+1}-2\psi_{i}+\psi_{i-1}}{\Delta^{2}},
$$

where $\Delta$, the distance between two mesh points, is assumed to be equidistant.

Now, instead of the isolated-box case ($\psi_{-1}=\psi_{n}=0$), we consider the following **periodic boundary conditions** (PBCs):
$$
\psi_{-1} = \psi_{n-1}, \quad \psi_{n} = \psi_{0}.
$$

**(Note: This will be extended later when we consider Bloch's theorem!)**

Then, at the end points ($i = 0$ and $i = n - 1$), we can write the second derivative as

$$
\left.\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}}\psi(x)\right|_{x = x_{0}} =
\frac{\psi_{-1}-2\psi_{0}+\psi_{2}}{\Delta^{2}} =
\frac{\textcolor{red}{\psi_{n-1}}-2\psi_{0}+\psi_{1}}{\Delta^{2}},
\\
\left.\frac{\mathrm{d}^{2}}{\mathrm{d}x^{2}}\psi(x)\right|_{x = x_{n-1}} =
\frac{\psi_{n-2}-2\psi_{n-1}+\psi_{n}}{\Delta^{2}} =
\frac{\psi_{n-2}-2\psi_{n-1}+\textcolor{red}{\psi_{0}}}{\Delta^{2}}.
$$

In matrix notation, the second-derivative can be then written as:

$$
\frac{1}{\Delta^{2}}
\begin{pmatrix}
-2 &  1 &    & & & & \color{red}{1} \\
 1 & -2 &  1\\
   &  1 & -2 &  1\\
   &    & \ddots & \ddots & \ddots \\
   &    &    &  1 & -2 &  1\\
   &    &    &    &  1 & -2 &  1\\
\color{red}{1} &    &    &    &    &  1 & -2\\
\end{pmatrix}
\begin{pmatrix}
\psi_{0}\\
\psi_{1}\\
\psi_{2}\\
\vdots\\
\psi_{n-3}\\
\psi_{n-2}\\
\psi_{n-1}
\end{pmatrix}.
$$

You can find the difference to the previous DBC kinetic energy matrix at the elements at $(0, n-1)$ and $(n-1, 0)$.  
Instead of 0, they are now 1.

The kinetic-energy matrix is then written as

$$
-\frac{1}{2 m \Delta^{2}}
\begin{pmatrix}
-2 &  1 &    & & & & \color{red}{1} \\
 1 & -2 &  1\\
   &  1 & -2 &  1\\
   &    & \ddots & \ddots & \ddots \\
   &    &    &  1 & -2 &  1\\
   &    &    &    &  1 & -2 &  1\\
\color{red}{1} &    &    &    &    &  1 & -2\\
\end{pmatrix}
\begin{pmatrix}
\psi_{0}\\
\psi_{1}\\
\psi_{2}\\
\vdots\\
\psi_{n-3}\\
\psi_{n-2}\\
\psi_{n-1}
\end{pmatrix}.
$$

Let's construct the matrix above using NumPy.

In [ ]:
# to_be_filled
s_PBC = System1D(xmin=0.0, xmax=1.0, nx=16, periodic=True)

Note that, with `periodic=True`, `xmax` is excluded from the mesh points, which is reasonable because we consider the PBCs, i.e., *ψ*(`xmin`) = *ψ*(`xmax`).

### Kinetic energy matrix

Write a function `calc_T_PBC(s)` to compute the kinetic-energy matrix for PBCs. You can do this, e.g., by modifying the one for the DBCs.

In [ ]:
# to_be_filled
def calc_T_PBC(s: System1D):
    """Calculate kinetic-energy matrix for PBCs in Hartree atomic units"""
    # 2nd derivative
    T = np.eye(s.nx, k=+1) - 2.0 * np.eye(s.nx) + np.eye(s.nx, k=-1)
    T[0, -1] = T[-1, 0] = 1.0  # PBCs
    T /= s.deltax**2

    # kinetic energy in Hartree atomic units (hbar = 1.0)
    T *= -1.0 / (2.0 * s.mass)

    return T

In [ ]:
# to_be_filled
s_PBC.T = calc_T_PBC(s_PBC)

### PBC vs DBC

Plot the obtained kinetic-energy matrix and compare to `s_DBC.T`

In [ ]:
# to_be_filled
vmax = np.abs(s_PBC.T).max()
vmin = -vmax
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12.8, 4.8))
ai1 = ax1.matshow(s_PBC.T, cmap='bwr', vmin=vmin, vmax=vmax)
ai2 = ax2.matshow(s_DBC.T, cmap='bwr', vmin=vmin, vmax=vmax)
fig.colorbar(ai1, ax=ax1)
fig.colorbar(ai2, ax=ax2)
ax1.set_title('PBCs')
ax2.set_title('DBCs')
plt.show()

### Schrödinger with PBCs

Solve the eigenproblem under the PBCs assuming no potential and plot the results.

In [ ]:
# to_be_filled

# potential energy (zero)
s_PBC.V = np.zeros((s_PBC.nx, s_PBC.nx))

# hamiltonian
s_PBC.H = s_PBC.T + s_PBC.V

# diagonalization
s_PBC.eigvals, s_PBC.eigvecs = np.linalg.eigh(s_PBC.H)

# normalization
s_PBC.psis = s_PBC.eigvecs / np.sqrt(s_PBC.deltax)

# densities
s_PBC.densities = s_PBC.psis**2

In [ ]:
# to_be_filled
plot_eigvals(s_PBC)

In [ ]:
# to_be_filled
interactive(
    plot_psi,
    s=fixed(s_PBC),
    nu=IntSlider(max=s_PBC.nx-1),
)

In [ ]:
# to_be_filled
interactive(
    plot_density,
    s=fixed(s_PBC),
    nu=IntSlider(max=s_PBC.nx-1),
)

Similarly to the Dirichlet boundary conditions, the analytical eigenvalues of the Schrödinger equation with periodic boundary conditions and without a potential are given by

$$
\epsilon_\nu = \frac{2 \nu^2 \pi^2 \hbar^2}{m a^2} \quad (\nu \in \mathbb{Z}).
$$

The difference is that $\nu$ can be now zero and negative. So we need to take the lowest energies for comparison, by sorting properly.

In [ ]:
# to_be_filled
s_PBC.nus = np.arange(-s_PBC.nx // 2, s_PBC.nx // 2)
s_PBC.nus

In [ ]:
# to_be_filled
s_PBC.eigvals_analytical = 2.0 * s_PBC.nus**2 * np.pi**2 / s_PBC.ax**2
s_PBC.eigvals_analytical = np.sort(s_PBC.eigvals_analytical)

In [ ]:
# to_be_filled
plot_eigvals_with_analytical(s_PBC)

Again, at higher eigenvalues, there is a large deviation.